In [1]:
from wikirag import WikiRAG
from langchain_ollama import ChatOllama
import pandas as pd
import time

In [2]:
num_doc_retrieved = 5
model_name = "llama3.1:8b"
basic_wikirag = WikiRAG(
    db_name = "./databases/Russia_2023_2024_full_4_table_rows_db", 
    model_name = model_name,
    documents_retrieved = num_doc_retrieved)
base_model = ChatOllama(model = basic_wikirag.model_name)

In [6]:
Q = [
    "When, where and how has Yevgeny Prigozhin died?",
    "Name all candidates who took part in Primorsky election. Where there any poles? Who had the best chances to win and who won?",
    "Have EU sanctions on Russia caused any troubles with strategic resources",
    "Tell me about things that doubt Russia policy reliablity in the eyes of democratic countries",
    "Tell me about Russia's economy in 2023 and 2024",
    "Who is the Minister of Defence of Russia?",
    "Who won 2024 Russian presidential election?",
    "Who won 2024 USA presidential election?"
]

In [7]:
base_model = ChatOllama(model = model_name)

In [8]:
df = pd.DataFrame(columns=[
    "question", 
    "answer", 
    "base model answer", 
    "retrivals", 
    "database search time",
    "retrival time",
    "model answer time",
    "base model asnwer time"
    ])

for question in Q:
    answer = basic_wikirag.query(question)
    
    times = basic_wikirag.get_time()
    start_time = time.time()
    base_model_answer = base_model.invoke(question)
    base_model_time = time.time() - start_time
    df = pd.concat([df, pd.DataFrame({
        "question": [question],
        "answer": [answer["answer"]],
        "base model answer": [base_model_answer.content], 
        "retrivals": [answer["retrivals"]],
        "database search time": [times["database search time"]],
        "retrival time": [times["retrival time"]],
        "model answer time": [times["model response time"]],
        "base model asnwer time": [base_model_time]
    })], ignore_index=True)

/tmp/ipykernel_5015/3466732227.py:19: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame({


In [9]:
df

,question,answer,base model answer,retrivals,database search time,retrival time,model answer time,base model asnwer time
0,"When, where and how has Yevgeny Prigozhin died?","According to the context, Yevgeny Prigozhin, a...",I don't have information about the death of Ye...,[2023 in Russia Deaths August 23 August\nYevg...,0.669753,6.970769,53.070965,7.459360
1,Name all candidates who took part in Primorsky...,Based on the context provided:\n\n**Candidates...,The Primorsky Krai legislative election took p...,[2023 Primorsky Krai gubernatorial election Ca...,0.441557,2.590772,55.186002,35.922039
2,Have EU sanctions on Russia caused any trouble...,"Based on the context, yes, EU sanctions on Rus...",The European Union's (EU) sanctions on Russia ...,[2022–2023 Russia–European Union gas dispute B...,0.821014,5.710029,22.515301,160.623319
3,Tell me about things that doubt Russia policy ...,"Based on the context, several events and state...",There are several factors that have contribute...,[2024 Russian presidential election Reactions ...,0.552101,4.301018,54.956142,104.605715
4,Tell me about Russia's economy in 2023 and 2024,"Based on the provided context, there is limite...","Economic forecasts are inherently uncertain, a...",[2024 in Russia Events March election: Incumb...,0.489133,4.401876,38.196980,181.895088
5,Who is the Minister of Defence of Russia?,The defence minister mentioned in the context ...,The current Minister of Defense of Russia is S...,[2024 in Russia Events May carrying the Kosmo...,0.360471,6.228308,12.057769,21.680492
6,Who won 2024 Russian presidential election?,"According to the context, Vladimir Putin won t...",I don't have any information about the outcome...,[2024 Russian presidential election Results ...,0.705690,5.030100,17.605889,19.246830
7,Who won 2024 USA presidential election?,The context provided does not mention the resu...,I don't have the ability to predict the future...,[Vladimir Putin 2024 presidential campaign Res...,0.559670,5.060609,29.685198,25.600280


In [11]:
df.to_csv("results.csv", sep = ";")